# Join table and relation in SQLAlchemy
***

The purpose of these laboratory classes is to familiarize with join table in SQLAlchemy.

The scope of this classes:
 - using join()
 - using outerjoin()

### Exercise

Use all of these methods to create queries for the test database. Check their execution time using the [profiling and timing code methods](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html).

For queries:
1. View a list of the names and surnames of managers living in the same country and working in the same store.
2. Find a list of all movies of the same length.
3. Find all clients living in the same city.


![schema dvd rental](dvd-rental-sample-database-diagram.png)

## Prepare the environment

From the previous we know how create query to database in SQLAlchemy based on function [select](https://docs.sqlalchemy.org/en/13/core/metadata.html?highlight=select#sqlalchemy.schema.Table.select) or [query](https://docs.sqlalchemy.org/en/14/orm/query.html)

To work properly in class, we will need the following configuration:

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

In [2]:
config_PostgreSQL = {
    "database_type": "",
    "user": "",
    "password": "",
    "database_url": "",
    "port": ,
    "database_name": ""
}

db_string = "{database_type}://{user}:{password}@{database_url}:{port}/{database_name}".format(**config_PostgreSQL)

engine = create_engine(db_string)

# test the connection
try:
    conn = engine.connect()
    print("Connected successfully!")
except Exception as e:
    print("Failed to connect")
    print(f"Error: {e}")

Base = declarative_base()

Connected successfully!


### Initialize mapper operation

We can use a script to initialize mapper operation. Where `dict_table` is the dictionary with tables representation where the key is the name of the table.

In [3]:
from sqlalchemy import MetaData, Table, select, Column, Integer, String, Date, ForeignKey, PrimaryKeyConstraint, select, and_
from sqlalchemy.inspection import inspect

metadata = MetaData()
dict_table = {}

inspector = inspect(engine)

for table_name in inspector.get_table_names():
    dict_table[table_name] = Table(table_name, metadata, autoload_with=engine)

The first part of the laboratory will concern the case of working with a database whose structure is don't well known.

In [4]:
from sqlalchemy.orm import sessionmaker

session = (sessionmaker(bind=engine))()

Base = declarative_base()

All the examples for this laboratory part will be for the tables that are mapped on the classes (Object representation).

This section presents issues related to the use of relationships described in table mapping classes. For a better understanding of the topic, a simple database will be created containing tables.

In [5]:
from sqlalchemy import Column, Integer, String, Date, ForeignKey
from sqlalchemy.orm import relationship, declarative_base

Base = declarative_base()

class Country(Base):
    __tablename__ = 'country'
    country_id = Column(Integer, primary_key=True)
    country = Column(String(50))
    last_update = Column(Date)
    # Relationships
    cities = relationship("City", back_populates="country")
    
    def __str__(self):
        return f"Country id: {self.country_id}, Country name: {self.country}, Last update: {self.last_update}"

class City(Base):
    __tablename__ = 'city'
    city_id = Column(Integer, primary_key=True)
    city = Column(String(50))
    country_id = Column(Integer, ForeignKey('country.country_id'))
    last_update = Column(Date)
    # Relationships
    country = relationship("Country", back_populates="cities")
    addresses = relationship("Address", back_populates="city")
    
    def __str__(self):
        return f"City id: {self.city_id}, City name: {self.city}, Country id: {self.country_id}, Last update: {self.last_update}"

class Address(Base):
    __tablename__ = 'address'
    address_id = Column(Integer, primary_key=True)
    address = Column(String(50))
    address2 = Column(String(50))
    district = Column(String(50))
    city_id = Column(Integer, ForeignKey('city.city_id'))
    postal_code = Column(String(10))
    phone = Column(String(50))
    last_update = Column(Date)
    # Relationships
    city = relationship("City", back_populates="addresses")
    customers = relationship("Customer", back_populates="address")
    staff_members = relationship("Staff", back_populates="address")
    stores = relationship("Store", back_populates="address")
    
    def __str__(self):
        return f"Address id: {self.address_id}, Address: {self.address}, City id: {self.city_id}, Last update: {self.last_update}"

class Staff(Base):
    __tablename__ = 'staff'
    staff_id = Column(Integer, primary_key=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    address_id = Column(Integer, ForeignKey('address.address_id'))
    email = Column(String(50))
    store_id = Column(Integer, ForeignKey('store.store_id'))
    active = Column(Integer)
    username = Column(String(50))
    password = Column(String(50))
    last_update = Column(Date)
    picture = Column(String)
    manager_staff_id = Column(Integer, ForeignKey('staff.staff_id'), nullable=True)
    
    # Relationships
    address = relationship("Address", back_populates="staff_members")
    store = relationship("Store", back_populates="staff_members", foreign_keys=[store_id])
    managed_store = relationship(
        "Store",
        back_populates="manager",
        primaryjoin="Store.manager_staff_id == Staff.staff_id",
        foreign_keys=[manager_staff_id]
    )

class Store(Base):
    __tablename__ = 'store'
    store_id = Column(Integer, primary_key=True)
    manager_staff_id = Column(Integer, ForeignKey('staff.staff_id'))
    address_id = Column(Integer, ForeignKey('address.address_id'))
    last_update = Column(Date)
    # Relationships
    address = relationship("Address", back_populates="stores")
    staff_members = relationship("Staff", back_populates="store", foreign_keys=[Staff.store_id])
    manager = relationship("Staff", back_populates="managed_store", foreign_keys=[Staff.manager_staff_id])
    customers = relationship("Customer", back_populates="store")
    
    def __str__(self):
        return f"Store id: {self.store_id}, Manager staff id: {self.manager_staff_id}, Address id: {self.address_id}, Last update: {self.last_update}"

class Film(Base):
    __tablename__ = 'film'
    film_id = Column(Integer, primary_key=True)
    title = Column(String(255))
    length = Column(Integer)
    last_update = Column(Date)
    # Relationships
    inventory_items = relationship("Inventory", back_populates="film")
    categories = relationship("Category", secondary='film_category', back_populates="films")

class Category(Base):
    __tablename__ = 'category'
    category_id = Column(Integer, primary_key=True)
    name = Column(String(25))
    last_update = Column(Date)
    # Relationships
    films = relationship("Film", secondary='film_category', back_populates="categories")

class Inventory(Base):
    __tablename__ = 'inventory'
    inventory_id = Column(Integer, primary_key=True)
    film_id = Column(Integer, ForeignKey('film.film_id'))
    store_id = Column(Integer, ForeignKey('store.store_id'))
    last_update = Column(Date)
    # Relationships
    film = relationship("Film", back_populates="inventory_items")
    store = relationship("Store", back_populates="inventory_items")
    rentals = relationship("Rental", back_populates="inventory")
    
    def __str__(self):
        return f"Inventory id: {self.inventory_id}, Film id: {self.film_id}, Store id: {self.store_id}, Last update: {self.last_update}"

class Rental(Base):
    __tablename__ = 'rental'
    rental_id = Column(Integer, primary_key=True)
    rental_date = Column(Date)
    inventory_id = Column(Integer, ForeignKey('inventory.inventory_id'))
    customer_id = Column(Integer, ForeignKey('customer.customer_id'))
    return_date = Column(Date)
    staff_id = Column(Integer, ForeignKey('staff.staff_id'))
    last_update = Column(Date)
    # Relationships
    inventory = relationship("Inventory", back_populates="rentals")
    customer = relationship("Customer", back_populates="rentals")
    staff = relationship("Staff", back_populates="rentals")
    
    def __str__(self):
        return f"Rental id: {self.rental_id}, Rental date: {self.rental_date}, Inventory id: {self.inventory_id}, Customer id: {self.customer_id}, Last update: {self.last_update}"

class Customer(Base):
    __tablename__ = 'customer'
    customer_id = Column(Integer, primary_key=True)
    store_id = Column(Integer, ForeignKey('store.store_id'))
    first_name = Column(String(45))
    last_name = Column(String(45))
    email = Column(String(50))
    address_id = Column(Integer, ForeignKey('address.address_id'))
    activebool = Column(Integer)
    create_date = Column(Date)
    last_update = Column(Date)
    active = Column(Integer)
    # Relationships
    address = relationship("Address", back_populates="customers")
    store = relationship("Store", back_populates="customers")
    rentals = relationship("Rental", back_populates="customer")
    
    def __str__(self):
        return f"Customer id: {self.customer_id}, Name: {self.first_name} {self.last_name}, Email: {self.email}, Store id: {self.store_id}, Last update: {self.last_update}"

## 1. View a list of the names and surnames of managers living in the same country and working in the same store.

To make join we can use script. As you can see, the join function creates queries that connect tables in a natural way (PK - FK relationship). But the query results will only appear for the columns contained in the table specified in the select or query functions.

Core API:

In [6]:
mapper_stmt1 = select(
    dict_table['staff'].c.first_name.label('manager_first_name'), 
    dict_table['staff'].c.last_name.label('manager_last_name'),
    dict_table['country'].c.country.label('country_name'),
    dict_table['store'].c.store_id.label('store_id')  # Jeśli masz nazwę sklepu jako kolumnę, możesz użyć tej kolumny
).select_from(
    dict_table['staff'].join(
        dict_table['store'], dict_table['staff'].c.staff_id == dict_table['store'].c.manager_staff_id
    ).join(
        dict_table['address'], dict_table['staff'].c.address_id == dict_table['address'].c.address_id
    ).join(
        dict_table['city'], dict_table['address'].c.city_id == dict_table['city'].c.city_id
    ).join(
        dict_table['country'], dict_table['city'].c.country_id == dict_table['country'].c.country_id
    )
).where(
    and_(
        dict_table['staff'].c.staff_id == dict_table['store'].c.manager_staff_id,
        dict_table['city'].c.country_id == dict_table['country'].c.country_id
    )
)

print('Mapper select: ')
print(mapper_stmt1)

with engine.connect() as conn:
    result1 = conn.execute(mapper_stmt1).fetchall()

print(result1)

Mapper select: 
SELECT staff.first_name AS manager_first_name, staff.last_name AS manager_last_name, country.country AS country_name, store.store_id AS store_id 
FROM staff JOIN store ON staff.staff_id = store.manager_staff_id JOIN address ON staff.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id 
WHERE staff.staff_id = store.manager_staff_id AND city.country_id = country.country_id
[('Mike', 'Hillyer', 'Canada', 1), ('Jon', 'Stephens', 'Australia', 2)]


ORM API:

In [7]:
session_stmt1 = session.query(
    dict_table['staff'].c.first_name.label('manager_first_name'),
    dict_table['staff'].c.last_name.label('manager_last_name'),
    dict_table['country'].c.country.label('country_name'),
    dict_table['store'].c.store_id.label('store_id')  # Jeśli masz nazwę sklepu jako kolumnę, użyj jej zamiast `store_id`
).join(
    dict_table['store'], dict_table['staff'].c.staff_id == dict_table['store'].c.manager_staff_id
).join(
    dict_table['address'], dict_table['staff'].c.address_id == dict_table['address'].c.address_id
).join(
    dict_table['city'], dict_table['address'].c.city_id == dict_table['city'].c.city_id
).join(
    dict_table['country'], dict_table['city'].c.country_id == dict_table['country'].c.country_id
).where(
    and_(
        dict_table['staff'].c.staff_id == dict_table['store'].c.manager_staff_id,
        dict_table['city'].c.country_id == dict_table['country'].c.country_id
    )
).order_by(dict_table['staff'].c.first_name.asc())

print('\nSession select: ')
print(session_stmt1)

result1 = session_stmt1.all()

print(result1)


Session select: 
SELECT staff.first_name AS manager_first_name, staff.last_name AS manager_last_name, country.country AS country_name, store.store_id AS store_id 
FROM staff JOIN store ON staff.staff_id = store.manager_staff_id JOIN address ON staff.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id 
WHERE staff.staff_id = store.manager_staff_id AND city.country_id = country.country_id ORDER BY staff.first_name ASC
[('Jon', 'Stephens', 'Australia', 2), ('Mike', 'Hillyer', 'Canada', 1)]


## 2. Find a list of all movies of the same length.

In [8]:
mapper_stmt2 = select(
    dict_table['film'].c.title.label('film_title'), 
    dict_table['film'].c.length.label('film_length')
).select_from(
    dict_table['film'].outerjoin(
        dict_table['film'].alias('f2'), dict_table['film'].c.length == dict_table['film'].alias('f2').c.length
    )
).where(
    dict_table['film'].c.length != None
).order_by(
    dict_table['film'].c.length.asc()
)

print('Mapper select for movies of the same length: ')
print(mapper_stmt2)

with engine.connect() as conn:
    result2 = conn.execute(mapper_stmt2).fetchall()

print(result2)

Mapper select for movies of the same length: 
SELECT film.title AS film_title, film.length AS film_length 
FROM film LEFT OUTER JOIN film AS f2 ON film.length = f2.length 
WHERE film.length IS NOT NULL ORDER BY film.length ASC
[('Ridgemont Submarine', 46), ('Ridgemont Submarine', 46), ('Ridgemont Submarine', 46), ('Ridgemont Submarine', 46), ('Ridgemont Submarine', 46), ('Iron Moon', 46), ('Iron Moon', 46), ('Iron Moon', 46), ('Iron Moon', 46), ('Iron Moon', 46), ('Alien Center', 46), ('Alien Center', 46), ('Alien Center', 46), ('Alien Center', 46), ('Alien Center', 46), ('Kwai Homeward', 46), ('Kwai Homeward', 46), ('Kwai Homeward', 46), ('Kwai Homeward', 46), ('Kwai Homeward', 46), ('Labyrinth League', 46), ('Labyrinth League', 46), ('Labyrinth League', 46), ('Labyrinth League', 46), ('Labyrinth League', 46), ('Divorce Shining', 47), ('Divorce Shining', 47), ('Divorce Shining', 47), ('Divorce Shining', 47), ('Divorce Shining', 47), ('Divorce Shining', 47), ('Divorce Shining', 47), ('

## 3. Find all clients living in the same city.

In [9]:
mapper_stmt3 = select(
    dict_table['customer'].c.first_name,
    dict_table['customer'].c.last_name,
    dict_table['city'].c.city
).select_from(
    dict_table['customer'].outerjoin(
        dict_table['address'], dict_table['customer'].c.address_id == dict_table['address'].c.address_id
    ).outerjoin(
        dict_table['city'], dict_table['address'].c.city_id == dict_table['city'].c.city_id
    )
)

print('Mapper select for clients in the same city (using outerjoin): ')
print(mapper_stmt3)

with engine.connect() as conn:
    result3 = conn.execute(mapper_stmt3).fetchall()

print(result3)

Mapper select for clients in the same city (using outerjoin): 
SELECT customer.first_name, customer.last_name, city.city 
FROM customer LEFT OUTER JOIN address ON customer.address_id = address.address_id LEFT OUTER JOIN city ON address.city_id = city.city_id
[('Jared', 'Ely', 'Purwakarta'), ('Mary', 'Smith', 'Sasebo'), ('Patricia', 'Johnson', 'San Bernardino'), ('Linda', 'Williams', 'Athenai'), ('Barbara', 'Jones', 'Myingyan'), ('Elizabeth', 'Brown', 'Nantou'), ('Jennifer', 'Davis', 'Laredo'), ('Maria', 'Miller', 'Kragujevac'), ('Susan', 'Wilson', 'Hamilton'), ('Margaret', 'Moore', 'Masqat'), ('Dorothy', 'Taylor', 'Esfahan'), ('Lisa', 'Anderson', 'Sagamihara'), ('Nancy', 'Thomas', 'Yamuna Nagar'), ('Karen', 'Jackson', 'Osmaniye'), ('Betty', 'White', 'Citrus Heights'), ('Helen', 'Harris', 'Bhopal'), ('Sandra', 'Martin', 'Southend-on-Sea'), ('Donna', 'Thompson', 'Elista'), ('Carol', 'Garcia', 'Kaduna'), ('Ruth', 'Martinez', 'Kimberley'), ('Sharon', 'Robinson', 'Mardan'), ('Michelle', 'Cl